In [1]:
from firedrake import *
import numpy as np
from petsc4py import *
from matplotlib import pyplot as plt


In [ ]:
def TrueSolution(n,T):
    mesh = UnitSquareMesh(n, n)
    
    order = 1
    V = FunctionSpace(mesh, "RT", order)
    Q = FunctionSpace(mesh, "DG", order-1)
    Z=V*Q
    Vhigh = FunctionSpace(mesh, "RT", order+2)
    Qhigh = FunctionSpace(mesh, "DG", order+1)
    
    VH = FunctionSpace(mesh, "CG", 1)

    up_ = Function(Z)
    up = Function(Z)
    
    vw = TestFunction(Z)
    
    n1 = 2.0 #make negative
    m1 = 2.0
    p1 = 1.0 #make small or 0
    q1 = 2.0 #make negative
    r1 = 2.0
    s1 = 1.0 #make small or 0
    
    # For this problem we need an initial condition::
    utrue = Function(Vhigh)
    utrue0_str = "-(p1*pi*cos(m1*pi*x[1])*sin(p1*pi*t)*sin(n1*pi*x[0]))"
    utrue1_str = "-(pi*s1*cos(pi*q1*x[0])*sin(pi*s1*t)*sin(pi*r1*x[1]))"
    ptrue_str = "-n1*pi*cos(p1*pi*t)*cos(n1*pi*x[0])*cos(m1*pi*x[1])-pi*r1*cos(pi*s1*t)*cos(pi*q1*x[0])*cos(pi*r1*x[1])"
    
    utrue_expr = Expression((utrue0_str, utrue1_str), p1=p1, m1=m1, n1=n1, s1=s1, q1=q1, r1=r1, t=0.0)
    
    ptrue = Function(Qhigh)
    ptrue_expr = Expression(ptrue_str, p1=p1, m1=m1, n1=n1, s1=s1, q1=q1, r1=r1, t=0.0)
    
    
    divphit = Function(Q)
    divphit_expr = Expression("-(n1*p1*pow(pi,2)*cos(n1*pi*x[0])*cos(m1*pi*x[1])*sin(p1*pi*t))-pow(pi,2)*r1*s1*cos(pi*q1*x[0])*cos(pi*r1*x[1])*sin(pi*s1*t)", p1=p1, m1=m1, n1=n1, s1=s1, q1=q1, r1=r1, t=0.0)
    
    ic = project(Expression((utrue0_str,utrue1_str,ptrue_str),p1=p1, m1=m1, n1=n1, s1=s1, q1=q1, r1=r1,t=0.0), Z)  #set to true solution to time t=0
    
    up_.assign(ic)
    up.assign(ic)
    
    u_, p_ = split(up_)
    u, p = split(up)
    v, w = split(vw)
    
    Theta = 0.5
    
    # Including the physical parameters
    dt = 1.0/n
    k = Constant(dt) 
    
    Forcing_expr = Expression(("(pi*((Beta*pow(n1,2) - pow(Eps,2)*pow(p1,2))*pi*cos(p1*pi*t)*cos(m1*pi*x[1])*sin(n1*pi*x[0]) - C*pow(Eps,2)*pow(H1,2)*p1*cos(m1*pi*x[1])*sin(p1*pi*t)*sin(n1*pi*x[0]) + Beta*pi*q1*r1*cos(pi*s1*t)*cos(pi*r1*x[1])*sin(pi*q1*x[0]) + Eps*s1*cos(pi*q1*x[0])*sin(pi*s1*t)*sin(pi*r1*x[1])))/pow(Eps,2)",
                               "-((pi*(Eps*p1*cos(m1*pi*x[1])*sin(p1*pi*t)*sin(n1*pi*x[0]) - Beta*m1*n1*pi*cos(p1*pi*t)*cos(n1*pi*x[0])*sin(m1*pi*x[1]) + cos(pi*q1*x[0])*((-(Beta*pi*pow(r1,2)) + pow(Eps,2)*pi*pow(s1,2))*cos(pi*s1*t) + C*pow(Eps,2)*pow(H1,2)*s1*sin(pi*s1*t))*sin(pi*r1*x[1])))/pow(Eps,2))"),Eps=Eps, Beta=Beta, H1=H1, n1=n1, m1=m1, p1=p1, q1=q1, r1=r1, s1=s1, C=C, t=0.0)
    
    Forcing = Function(V)
    
    #Forcing_eta_expr = Expression(("(-(n1*p1*pow(pi,2)*cos(n1*pi*x[0])*cos(m1*pi*x[1])*sin(p1*pi*t))-pow(pi,2)*r1*s1*cos(pi*q1*x[0])*cos(pi*r1*x[1])*sin(pi*s1*t))*2"),n1=n1, m1=m1, p1=p1, q1=q1, r1=r1, s1=s1, t=0.0)
    #Forcing_eta = Function(Q)
    
    F = (
        (inner(v,(u-u_)))*dx
        - (inner(div(v),(Theta*p+(1-Theta)*p_))*k)*dx
        - (inner(w,(p-p_)))*dx
        - (inner(w,div(Theta*u+(1-Theta)*u_))*k)*dx
        - (inner(Forcing.project(Forcing_expr),v)*k)*dx
        #- (inner(Forcing_eta.project(Forcing_eta_expr),w)*k)*dx
    )
    
    solver_params={
        "mat_type": "aij",
        "ksp_type": "preonly",
        "pc_type": "lu",
        "snes_linesearch_type": "basic",
        #"snes_monitor": None
    }
    
    bcs = [DirichletBC(Z.sub(0), 0, (1,2,3,4))]
    time_array=[]
    energy_array=[]
    
    t = 0.0
    
    uu, pp = TrialFunctions(Z)
    Jpc = (
        inner(v,uu*Hinv)*dx
        + (inner(v,perp(Theta*uu))*Hinv/Eps*k)*dx
        - (Beta/Eps/Eps*div(v)*(Theta*pp)*k)*dx
        + ((w*(pp))*dx
           +(w*div(Theta*uu)*k)*dx)
    )
    
    
    uerror_array = []
    perror_array = []
    divphit_array = []
    
    while (t <= T):
        Forcing_expr.t = t+.5*dt
        utrue_expr.t = t+dt
        ptrue_expr.t = t+dt
        #divphit_expr.t = t+dt
        
        Forcing.project(Forcing_expr)
        
        time_array.append(t)
        #energy_array.append(energy)
        solve(F == 0, up, bcs=bcs,
              J=Jpc,
              solver_parameters=solver_params)
        up_.assign(up)
        
        u, p = up.split()
        
        utrue.project(utrue_expr)
        ptrue.project(ptrue_expr)
        
        uerror = sqrt(assemble(inner(u-utrue,u-utrue)*dx))
        uerror_array.append(uerror)
        
        perror = sqrt(assemble(inner(p-ptrue,p-ptrue)*dx))
        perror_array.append(perror)
        
        #divphiterror = assemble(divphit.project(divphit_expr)*dx)
        #divphit_array.append(divphiterror)
        
        print "%1.3e\t%e %e %e" % (t, uerror, perror, assemble(p_*dx))
        
        t += dt
        

    #import matplotlib.pyplot as plt
        
    #plt.plot(time_array,energy_array)
    #plt.plot(time_array,uerror_array)
    #plt.plot(time_array,perror_array)
    #plt.show()
    #plt.close()


if __name__ == "__main__":
    n = 20
    T = 10.0
    TrueSolution(n,T)
